In [1]:
#Import - Library

import pandas as pd
import numpy as np
import requests as req

In [2]:
#Import - Output of Data Scraping

rdf = pd.read_csv('Zomato Restaurants DataFrame - No Duplicate - Final.csv', index_col=False)

In [3]:
#Make A New DataFrame - for only important features that will be used for Reverse Geocoding

rdf_geo = rdf[['Restaurant Name', 'Restaurant Area', 'Restaurant Address', 'Latitude', 'Longitude']]

In [4]:
#Show RDF_GEO

rdf_geo

,Restaurant Name,Restaurant Area,Restaurant Address,Latitude,Longitude
0,Holy Smokes,Senopati,"Jl. Wolter Monginsidi No. 27, Senopati, Jakarta",-6.2391432684,106.807954162
1,Pempek Megaria,Cikini,"Metropole 21, Jl. Pegangsaan Timur No. 21, Cik...",-6.1999000000,106.843739000
2,Tjikinii Lima,Cikini,"Jl. Cikini I No. 5, Cikini, Jakarta 10330",-6.1898019937,106.837150007
3,Gado-Gado Bon-Bin,Cikini,"Jl. Cikini 4 No. 5, Cikini, Jakarta",-6.1929590000,106.838377000
4,Koffie Fictie,Bekasi Selatan,"Grand Galaxy Park, Ruko RSOD No. 026, Jl. Lotu...",-6.2720590000,106.970932000
...,...,...,...,...,...
11262,Essence Coffee,Pondok Bambu,"Jl. Bambu Ori Raya, Pondok Bambu, Jakarta",-6.2376240000,106.907457000
11263,Senja Toast,Jagakarsa,"Jl. Raya Lenteng Agung, Jagakarsa, Jakarta",-6.3443890000,106.832833000
11264,Express Juice,Bojongsari,"Jl. Raya Ciputat Parung no 60, Bojongsari, Depok",-6.3611110000,106.747747000
11265,Ayam Selebriti,Meruya,"Jl. Meruya Selatan No.21, Meruya, Jakarta",-6.2109790000,106.737538000


# WHY WE NEED REVERSE GEOCODING

In [6]:
#Check Empty Value in Restaurant Area

rdf['Restaurant Area'].isna().sum()

0

In [ ]:
#Check unique values of Restaurant Area

rdf['Restaurant Area'].nunique()

In [7]:
rdf['Restaurant Area'].unique()

array(['Senopati', 'Cikini', 'Bekasi Selatan', 'Setiabudi', 'Serpong',
       'Thamrin', 'Cinere', 'Gunung Sahari', 'Karet', 'Larangan',
       'Kuningan', 'Mangga Dua', 'Meruya', 'Pondok Aren', 'Kelapa Gading',
       'Kampung Melayu', 'Beji', 'Medan Satria', 'Tanjung Duren',
       'Sunter', 'Mangga Besar', 'Cimanggis', 'Daan Mogot', 'Grogol',
       'Pinang Ranti', 'Bogor Tengah', 'Pluit', 'Bogor Utara',
       'Bekasi Utara', 'Fatmawati', 'Cempaka Putih', 'Cibubur',
       'Pondok Indah', 'Bogor Timur', 'Pasar Minggu', 'Serpong Utara',
       'Ciputat Timur', 'Rawalumbu', 'Roxy', 'Gambir', 'Kemayoran',
       'Jatiuwung', 'Lebak Bulus', 'Ancol', 'Matraman', 'Jagakarsa',
       'Bintaro', 'Tanah Sareal', 'Bendungan Hilir', 'Kec. Tangerang',
       'Cipayung', 'Cibodas', 'Pondok Bambu', 'Tanah Abang', 'Cawang',
       'Cipondoh', 'Cakung', 'Senen', 'Jatisampurna', 'Senayan', 'Tebet',
       'Sawangan', 'Cengkareng', 'Taman Mini', 'Dharmawangsa', 'Kemang',
       'Sukmajaya', 'SCBD', 

In [ ]:
In Order to better understand our Data (and to make better visualization later), it's better for us to do reverse geocoding because:

1. Zomato Area Feature actually show the area of restaurant without the same level of detail, the easiest example would be: there is SCBD Area, Gatot Subroto Area, while other data are Cawang Tebet.
It's not the same level of detail, and we need the same level of detail when we do analysis later, maybe from the 'Kecamatan' Level of Detail or 'Province' Level of Detail

2. Zomato doesn't specify which Area is in DKI Jakarta (we need to infer it ourselves by checking area/address one by one), or even Jakarta Utara, Timur, Selatan, Barat, etc.

# MAKING SURE WE CAN REQUEST REVERESE GEOCODING CORRECTLY

In order to succesfully request geocoding data, we need to make sure all the Latitude an Longitude are written correctly.

Let's check them!

In Indonesia, Latitude started with -x.xxxx

In [8]:
rdf[rdf['Latitude'].str.slice(0,1) != '-']

,Restaurant Name,Restaurant Type,Cuisine Type,Restaurant Area,Restaurant Rating,Review Counts,Latitude,Longitude,Website,Restaurant Address,Price for 2,Additional Info
883,Geprek Gold Chick,Quick Bites,['Indonesia'],Penjaringan,Not Rated Yet,TIdak cukup Ulasan,s/dir/?api=1&,"estination=0,",https://www.zomato.com/id/jakarta/geprek-gold-...,"Jl. Teluk Gong Raya, Penjaringan, Jakarta",Rp50.000 untuk 2 orang (perkiraan),"['Bawa Pulang Tersedia', 'Di dalam ruangan']"
2786,Yokobento,Quick Bites,['Jepang'],Mustikajaya,3.2,7 Ulasan,on=0.06392038,"3,0.000000000",https://www.zomato.com/id/jakarta/yokobento-mu...,"Ruko Duku Zamrud, Blok N, No. 19, Jl. Ruko Duk...",Rp70.000 untuk 2 orang (perkiraan),"['Sarapan', 'Pesan Antar', 'Bawa Pulang Tersed..."
2925,Nasi Uduk Jayadi,Kaki Lima,['Indonesia'],Bintaro,Not Rated Yet,Not Reviewed Yet,s/dir/?api=1&,"estination=0,",https://www.zomato.com/id/jakarta/nasi-uduk-ja...,"Jl. Pondok Betung Raya No. 8, Bintaro, Jakarta",Rp50.000 untuk 2 orang (perkiraan),"['Bawa Pulang Tersedia', 'Tempat duduk di luar..."
3238,Bopan Coffee,Toko Minuman,"['Minuman', 'Tea', 'Kopi']",Daan Mogot,Not Rated Yet,Not Reviewed Yet,s/dir/?api=1&,"estination=0,",https://www.zomato.com/id/jakarta/bopan-coffee...,"Ruko Seventh Esplanade B02/7, Citra Garden Cit...",Rp50 untuk 2 orang (perkiraan),"['Bawa Pulang Tersedia', 'Di dalam ruangan']"
3699,Serba Nikmat Bakmi & Chinese Food,Quick Bites,"['Makanan Cepat Saji', 'Chinese']",Gajah Mada,Not Rated Yet,Not Reviewed Yet,s/dir/?api=1&,"estination=0,",https://www.zomato.com/id/jakarta/serba-nikmat...,"Jl. 213 B, RT 1/RW 1, Glodok, Kec, Taman Sari,...",Rp70 untuk 2 orang (perkiraan),"['Bawa Pulang Tersedia', 'Di dalam ruangan']"
8621,Kopimen,Quick Bites,"['Makanan Jalanan', 'Kopi', 'Tea']",Palmerah,3.1,16 Ulasan,=6.2007500000,106.785232000,https://www.zomato.com/id/jakarta/kopimen-palm...,"Jl. KH. Syahdan No. 1, Palmerah, Jakarta 11480",Rp120.000 untuk 2 orang (perkiraan),"['Bawa Pulang Tersedia', 'Di dalam ruangan', '..."


..... and the Longitude started with 1.xxxx

In [9]:
rdf[rdf['Longitude'].str.slice(0,1) != '1']

,Restaurant Name,Restaurant Type,Cuisine Type,Restaurant Area,Restaurant Rating,Review Counts,Latitude,Longitude,Website,Restaurant Address,Price for 2,Additional Info
883,Geprek Gold Chick,Quick Bites,['Indonesia'],Penjaringan,Not Rated Yet,TIdak cukup Ulasan,s/dir/?api=1&,"estination=0,",https://www.zomato.com/id/jakarta/geprek-gold-...,"Jl. Teluk Gong Raya, Penjaringan, Jakarta",Rp50.000 untuk 2 orang (perkiraan),"['Bawa Pulang Tersedia', 'Di dalam ruangan']"
2786,Yokobento,Quick Bites,['Jepang'],Mustikajaya,3.2,7 Ulasan,on=0.06392038,"3,0.000000000",https://www.zomato.com/id/jakarta/yokobento-mu...,"Ruko Duku Zamrud, Blok N, No. 19, Jl. Ruko Duk...",Rp70.000 untuk 2 orang (perkiraan),"['Sarapan', 'Pesan Antar', 'Bawa Pulang Tersed..."
2925,Nasi Uduk Jayadi,Kaki Lima,['Indonesia'],Bintaro,Not Rated Yet,Not Reviewed Yet,s/dir/?api=1&,"estination=0,",https://www.zomato.com/id/jakarta/nasi-uduk-ja...,"Jl. Pondok Betung Raya No. 8, Bintaro, Jakarta",Rp50.000 untuk 2 orang (perkiraan),"['Bawa Pulang Tersedia', 'Tempat duduk di luar..."
3238,Bopan Coffee,Toko Minuman,"['Minuman', 'Tea', 'Kopi']",Daan Mogot,Not Rated Yet,Not Reviewed Yet,s/dir/?api=1&,"estination=0,",https://www.zomato.com/id/jakarta/bopan-coffee...,"Ruko Seventh Esplanade B02/7, Citra Garden Cit...",Rp50 untuk 2 orang (perkiraan),"['Bawa Pulang Tersedia', 'Di dalam ruangan']"
3699,Serba Nikmat Bakmi & Chinese Food,Quick Bites,"['Makanan Cepat Saji', 'Chinese']",Gajah Mada,Not Rated Yet,Not Reviewed Yet,s/dir/?api=1&,"estination=0,",https://www.zomato.com/id/jakarta/serba-nikmat...,"Jl. 213 B, RT 1/RW 1, Glodok, Kec, Taman Sari,...",Rp70 untuk 2 orang (perkiraan),"['Bawa Pulang Tersedia', 'Di dalam ruangan']"


In [10]:
faulty_data = [883, 2786, 2925, 3238, 3699, 8621]

After we check one by one to Zomato Website, turned out their pages are really showing no / wrong maps
So let's get their latitude/longitude individually by Gmaps

In [11]:
rdf.loc[883]['Latitude'] = '-6.1383350000'
rdf.loc[883]['Longitude'] = '106.7817120000'

rdf.loc[2786]['Latitude'] = '-6.3104988'
rdf.loc[2786]['Longitude'] = '107.0199968'

rdf.loc[2925]['Latitude'] = '-6.2782715'
rdf.loc[2925]['Longitude'] = '106.7508656'

rdf.loc[3238]['Latitude'] = '-6.1468311'
rdf.loc[3238]['Longitude'] = '106.6988851'

rdf.loc[3699]['Latitude'] = '-6.1447486'
rdf.loc[3699]['Longitude'] = '106.8130754'

rdf.loc[8621]['Latitude'] = '-6.2007560000'
rdf.loc[8621]['Longitude'] = '106.7852540000'

In [12]:
#Let's check the result!

rdf.loc[faulty_data]

,Restaurant Name,Restaurant Type,Cuisine Type,Restaurant Area,Restaurant Rating,Review Counts,Latitude,Longitude,Website,Restaurant Address,Price for 2,Additional Info
883,Geprek Gold Chick,Quick Bites,['Indonesia'],Penjaringan,Not Rated Yet,TIdak cukup Ulasan,-6.1383350000,106.7817120000,https://www.zomato.com/id/jakarta/geprek-gold-...,"Jl. Teluk Gong Raya, Penjaringan, Jakarta",Rp50.000 untuk 2 orang (perkiraan),"['Bawa Pulang Tersedia', 'Di dalam ruangan']"
2786,Yokobento,Quick Bites,['Jepang'],Mustikajaya,3.2,7 Ulasan,-6.3104988,107.0199968,https://www.zomato.com/id/jakarta/yokobento-mu...,"Ruko Duku Zamrud, Blok N, No. 19, Jl. Ruko Duk...",Rp70.000 untuk 2 orang (perkiraan),"['Sarapan', 'Pesan Antar', 'Bawa Pulang Tersed..."
2925,Nasi Uduk Jayadi,Kaki Lima,['Indonesia'],Bintaro,Not Rated Yet,Not Reviewed Yet,-6.2782715,106.7508656,https://www.zomato.com/id/jakarta/nasi-uduk-ja...,"Jl. Pondok Betung Raya No. 8, Bintaro, Jakarta",Rp50.000 untuk 2 orang (perkiraan),"['Bawa Pulang Tersedia', 'Tempat duduk di luar..."
3238,Bopan Coffee,Toko Minuman,"['Minuman', 'Tea', 'Kopi']",Daan Mogot,Not Rated Yet,Not Reviewed Yet,-6.1468311,106.6988851,https://www.zomato.com/id/jakarta/bopan-coffee...,"Ruko Seventh Esplanade B02/7, Citra Garden Cit...",Rp50 untuk 2 orang (perkiraan),"['Bawa Pulang Tersedia', 'Di dalam ruangan']"
3699,Serba Nikmat Bakmi & Chinese Food,Quick Bites,"['Makanan Cepat Saji', 'Chinese']",Gajah Mada,Not Rated Yet,Not Reviewed Yet,-6.1447486,106.8130754,https://www.zomato.com/id/jakarta/serba-nikmat...,"Jl. 213 B, RT 1/RW 1, Glodok, Kec, Taman Sari,...",Rp70 untuk 2 orang (perkiraan),"['Bawa Pulang Tersedia', 'Di dalam ruangan']"
8621,Kopimen,Quick Bites,"['Makanan Jalanan', 'Kopi', 'Tea']",Palmerah,3.1,16 Ulasan,-6.2007560000,106.7852540000,https://www.zomato.com/id/jakarta/kopimen-palm...,"Jl. KH. Syahdan No. 1, Palmerah, Jakarta 11480",Rp120.000 untuk 2 orang (perkiraan),"['Bawa Pulang Tersedia', 'Di dalam ruangan', '..."


In [13]:
rdf.to_excel('Zomato Restaurant DataFrame - LatLong Correct.xlsx', index = False)
rdf.to_csv('Zomato Restaurant DataFrame - LatLong Correct.csv', index = False)

# REVERSE GEOCODING DATA will USE HERE MAPS API

# STEP 1 - TEST REVERSE GEOCODING using 5 ROWS of DATA
### I do this test because collecting data through API has it's price, so before we request too much, let's check by using small quantity of data first

In [62]:
rdf_geo5 = rdf_geo[0:5]

In [63]:
rdf_geo5

,Restaurant Name,Restaurant Area,Restaurant Address,Latitude,Longitude
0,Holy Smokes,Senopati,"Jl. Wolter Monginsidi No. 27, Senopati, Jakarta",-6.239143,106.807954
1,Pempek Megaria,Cikini,"Metropole 21, Jl. Pegangsaan Timur No. 21, Cik...",-6.199900,106.843739
2,Tjikinii Lima,Cikini,"Jl. Cikini I No. 5, Cikini, Jakarta 10330",-6.189802,106.837150
3,Gado-Gado Bon-Bin,Cikini,"Jl. Cikini 4 No. 5, Cikini, Jakarta",-6.192959,106.838377
4,Koffie Fictie,Bekasi Selatan,"Grand Galaxy Park, Ruko RSOD No. 026, Jl. Lotu...",-6.272059,106.970932


In [69]:
URL = "https://revgeocode.search.hereapi.com/v1/revgeocode"
api_key = 'ZEB4Ggb6IZp2p7DnmBpUmnk-sR5-Ih5HhrVrIJHaGkQ'

# Create Empty List to Store Geodata
geodata = []


for index, row in rdf_geo5.iterrows():

    latitude = row['Latitude']
    longitude = row['Longitude']

    # Defining a params dictionary for the parameters to be sent to the API 
    PARAMS = {
			    'at': '{},{}'.format(latitude,longitude),
			    'apikey': api_key
            }

    # Sending get request and saving the response as response object 
    r = req.get(url = URL, params = PARAMS) 
  
    # Extracting data in json format 
    data = r.json() 

    # Appending data to geodata
    geodata.append(data)

In [70]:
geodata

[{'items': [{'title': 'SMK Penerbangan PGRI',
    'id': 'here:pds:place:360qqguq-a96ddfa855cd444183093b22f6246126',
    'resultType': 'place',
    'address': {'label': 'SMK Penerbangan PGRI, Jalan Wolter Monginsidi, Jakarta Selatan 12170, Indonesia',
     'countryCode': 'IDN',
     'countryName': 'Indonesia',
     'state': 'JABODETABEK',
     'county': 'DKI Jakarta',
     'city': 'Jakarta Selatan',
     'district': 'Kebayoran Baru',
     'subdistrict': 'Petogogan Kel.',
     'street': 'Jalan Wolter Monginsidi',
     'postalCode': '12170'},
    'position': {'lat': -6.23919, 'lng': 106.80785},
    'access': [{'lat': -6.23914, 'lng': 106.80785}],
    'distance': 12,
    'categories': [{'id': '800-8200-0174',
      'name': 'Sekolah',
      'primary': True}]}]},
 {'items': [{'title': 'Somay Dan es Teler Megaria',
    'id': 'here:pds:place:360qqgux-b1b4fa364b8cec902c380bf8e7f8c435',
    'resultType': 'place',
    'address': {'label': 'Somay Dan es Teler Megaria, Jakarta Pusat 10320, Indonesi

In [74]:
#Create - Empty List to store the Data that we need from JSON geodata

state_list = []
county_list = []
city_list = []
district_list = []

#Loop Through - to fulfill the Data List

for i in range(0,5):
    state_list.append(geodata[i]['items'][0]['address']['state'])
    county_list.append(geodata[i]['items'][0]['address']['county'])
    city_list.append(geodata[i]['items'][0]['address']['city'])
    district_list.append(geodata[i]['items'][0]['address']['district'])

In [85]:
#Make A New DataFrame - by combining previous list

rdf_revgeo = pd.DataFrame({'State': state_list, 'Province': county_list, 'City': city_list, 'District/Kecamatan': district_list })

In [86]:
rdf_revgeo

,State,Province,City,District/Kecamatan
0,JABODETABEK,DKI Jakarta,Jakarta Selatan,Kebayoran Baru
1,JABODETABEK,DKI Jakarta,Jakarta Pusat,Menteng
2,JABODETABEK,DKI Jakarta,Jakarta Pusat,Menteng
3,JABODETABEK,DKI Jakarta,Jakarta Pusat,Menteng
4,JABODETABEK,Jawa Barat,Bekasi Kota,Bekasi Selatan


In [87]:
rdf_geo_comp = pd.merge(left=rdf_geo5, right=rdf_revgeo, left_index=True, right_index=True) 

In [88]:
rdf_geo_comp

,Restaurant Name,Restaurant Area,Restaurant Address,Latitude,Longitude,State,Province,City,District/Kecamatan
0,Holy Smokes,Senopati,"Jl. Wolter Monginsidi No. 27, Senopati, Jakarta",-6.239143,106.807954,JABODETABEK,DKI Jakarta,Jakarta Selatan,Kebayoran Baru
1,Pempek Megaria,Cikini,"Metropole 21, Jl. Pegangsaan Timur No. 21, Cik...",-6.199900,106.843739,JABODETABEK,DKI Jakarta,Jakarta Pusat,Menteng
2,Tjikinii Lima,Cikini,"Jl. Cikini I No. 5, Cikini, Jakarta 10330",-6.189802,106.837150,JABODETABEK,DKI Jakarta,Jakarta Pusat,Menteng
3,Gado-Gado Bon-Bin,Cikini,"Jl. Cikini 4 No. 5, Cikini, Jakarta",-6.192959,106.838377,JABODETABEK,DKI Jakarta,Jakarta Pusat,Menteng
4,Koffie Fictie,Bekasi Selatan,"Grand Galaxy Park, Ruko RSOD No. 026, Jl. Lotu...",-6.272059,106.970932,JABODETABEK,Jawa Barat,Bekasi Kota,Bekasi Selatan


# STEP 2 - REVERSE GEOCODING using COMPLETE DATA

In [7]:
URL = "https://revgeocode.search.hereapi.com/v1/revgeocode"
api_key = 'ZEB4Ggb6IZp2p7DnmBpUmnk-sR5-Ih5HhrVrIJHaGkQ'

# Create Empty List to store geodata
geodata = []
count = 0


for index, row in rdf_geo.iterrows():

    latitude = row['Latitude']
    longitude = row['Longitude']

    # Defining a params dictionary for the parameters to be sent to the API 
    PARAMS = {
			    'at': '{},{}'.format(latitude,longitude),
			    'apikey': api_key
            }

    # Sending get request and saving the response as response object 
    r = req.get(url = URL, params = PARAMS) 
  
    # Extracting data in json format 
    data = r.json() 

    # Appending data to geodata
    geodata.append(data)

    count += 1
    print("Get Data {}".format(count))

Get Data 1
Get Data 2
Get Data 3
Get Data 4
Get Data 5
Get Data 6
Get Data 7
Get Data 8
Get Data 9
Get Data 10
Get Data 11
Get Data 12
Get Data 13
Get Data 14
Get Data 15
Get Data 16
Get Data 17
Get Data 18
Get Data 19
Get Data 20
Get Data 21
Get Data 22
Get Data 23
Get Data 24
Get Data 25
Get Data 26
Get Data 27
Get Data 28
Get Data 29
Get Data 30
Get Data 31
Get Data 32
Get Data 33
Get Data 34
Get Data 35
Get Data 36
Get Data 37
Get Data 38
Get Data 39
Get Data 40
Get Data 41
Get Data 42
Get Data 43
Get Data 44
Get Data 45
Get Data 46
Get Data 47
Get Data 48
Get Data 49
Get Data 50
Get Data 51
Get Data 52
Get Data 53
Get Data 54
Get Data 55
Get Data 56
Get Data 57
Get Data 58
Get Data 59
Get Data 60
Get Data 61
Get Data 62
Get Data 63
Get Data 64
Get Data 65
Get Data 66
Get Data 67
Get Data 68
Get Data 69
Get Data 70
Get Data 71
Get Data 72
Get Data 73
Get Data 74
Get Data 75
Get Data 76
Get Data 77
Get Data 78
Get Data 79
Get Data 80
Get Data 81
Get Data 82
Get Data 83
Get Data 84
G

Get Data 640
Get Data 641
Get Data 642
Get Data 643
Get Data 644
Get Data 645
Get Data 646
Get Data 647
Get Data 648
Get Data 649
Get Data 650
Get Data 651
Get Data 652
Get Data 653
Get Data 654
Get Data 655
Get Data 656
Get Data 657
Get Data 658
Get Data 659
Get Data 660
Get Data 661
Get Data 662
Get Data 663
Get Data 664
Get Data 665
Get Data 666
Get Data 667
Get Data 668
Get Data 669
Get Data 670
Get Data 671
Get Data 672
Get Data 673
Get Data 674
Get Data 675
Get Data 676
Get Data 677
Get Data 678
Get Data 679
Get Data 680
Get Data 681
Get Data 682
Get Data 683
Get Data 684
Get Data 685
Get Data 686
Get Data 687
Get Data 688
Get Data 689
Get Data 690
Get Data 691
Get Data 692
Get Data 693
Get Data 694
Get Data 695
Get Data 696
Get Data 697
Get Data 698
Get Data 699
Get Data 700
Get Data 701
Get Data 702
Get Data 703
Get Data 704
Get Data 705
Get Data 706
Get Data 707
Get Data 708
Get Data 709
Get Data 710
Get Data 711
Get Data 712
Get Data 713
Get Data 714
Get Data 715
Get Data 716

Get Data 1251
Get Data 1252
Get Data 1253
Get Data 1254
Get Data 1255
Get Data 1256
Get Data 1257
Get Data 1258
Get Data 1259
Get Data 1260
Get Data 1261
Get Data 1262
Get Data 1263
Get Data 1264
Get Data 1265
Get Data 1266
Get Data 1267
Get Data 1268
Get Data 1269
Get Data 1270
Get Data 1271
Get Data 1272
Get Data 1273
Get Data 1274
Get Data 1275
Get Data 1276
Get Data 1277
Get Data 1278
Get Data 1279
Get Data 1280
Get Data 1281
Get Data 1282
Get Data 1283
Get Data 1284
Get Data 1285
Get Data 1286
Get Data 1287
Get Data 1288
Get Data 1289
Get Data 1290
Get Data 1291
Get Data 1292
Get Data 1293
Get Data 1294
Get Data 1295
Get Data 1296
Get Data 1297
Get Data 1298
Get Data 1299
Get Data 1300
Get Data 1301
Get Data 1302
Get Data 1303
Get Data 1304
Get Data 1305
Get Data 1306
Get Data 1307
Get Data 1308
Get Data 1309
Get Data 1310
Get Data 1311
Get Data 1312
Get Data 1313
Get Data 1314
Get Data 1315
Get Data 1316
Get Data 1317
Get Data 1318
Get Data 1319
Get Data 1320
Get Data 1321
Get Da

Get Data 1837
Get Data 1838
Get Data 1839
Get Data 1840
Get Data 1841
Get Data 1842
Get Data 1843
Get Data 1844
Get Data 1845
Get Data 1846
Get Data 1847
Get Data 1848
Get Data 1849
Get Data 1850
Get Data 1851
Get Data 1852
Get Data 1853
Get Data 1854
Get Data 1855
Get Data 1856
Get Data 1857
Get Data 1858
Get Data 1859
Get Data 1860
Get Data 1861
Get Data 1862
Get Data 1863
Get Data 1864
Get Data 1865
Get Data 1866
Get Data 1867
Get Data 1868
Get Data 1869
Get Data 1870
Get Data 1871
Get Data 1872
Get Data 1873
Get Data 1874
Get Data 1875
Get Data 1876
Get Data 1877
Get Data 1878
Get Data 1879
Get Data 1880
Get Data 1881
Get Data 1882
Get Data 1883
Get Data 1884
Get Data 1885
Get Data 1886
Get Data 1887
Get Data 1888
Get Data 1889
Get Data 1890
Get Data 1891
Get Data 1892
Get Data 1893
Get Data 1894
Get Data 1895
Get Data 1896
Get Data 1897
Get Data 1898
Get Data 1899
Get Data 1900
Get Data 1901
Get Data 1902
Get Data 1903
Get Data 1904
Get Data 1905
Get Data 1906
Get Data 1907
Get Da

Get Data 2423
Get Data 2424
Get Data 2425
Get Data 2426
Get Data 2427
Get Data 2428
Get Data 2429
Get Data 2430
Get Data 2431
Get Data 2432
Get Data 2433
Get Data 2434
Get Data 2435
Get Data 2436
Get Data 2437
Get Data 2438
Get Data 2439
Get Data 2440
Get Data 2441
Get Data 2442
Get Data 2443
Get Data 2444
Get Data 2445
Get Data 2446
Get Data 2447
Get Data 2448
Get Data 2449
Get Data 2450
Get Data 2451
Get Data 2452
Get Data 2453
Get Data 2454
Get Data 2455
Get Data 2456
Get Data 2457
Get Data 2458
Get Data 2459
Get Data 2460
Get Data 2461
Get Data 2462
Get Data 2463
Get Data 2464
Get Data 2465
Get Data 2466
Get Data 2467
Get Data 2468
Get Data 2469
Get Data 2470
Get Data 2471
Get Data 2472
Get Data 2473
Get Data 2474
Get Data 2475
Get Data 2476
Get Data 2477
Get Data 2478
Get Data 2479
Get Data 2480
Get Data 2481
Get Data 2482
Get Data 2483
Get Data 2484
Get Data 2485
Get Data 2486
Get Data 2487
Get Data 2488
Get Data 2489
Get Data 2490
Get Data 2491
Get Data 2492
Get Data 2493
Get Da

...

Get Data 8869
Get Data 8870
Get Data 8871
Get Data 8872
Get Data 8873
Get Data 8874
Get Data 8875
Get Data 8876
Get Data 8877
Get Data 8878
Get Data 8879
Get Data 8880
Get Data 8881
Get Data 8882
Get Data 8883
Get Data 8884
Get Data 8885
Get Data 8886
Get Data 8887
Get Data 8888
Get Data 8889
Get Data 8890
Get Data 8891
Get Data 8892
Get Data 8893
Get Data 8894
Get Data 8895
Get Data 8896
Get Data 8897
Get Data 8898
Get Data 8899
Get Data 8900
Get Data 8901
Get Data 8902
Get Data 8903
Get Data 8904
Get Data 8905
Get Data 8906
Get Data 8907
Get Data 8908
Get Data 8909
Get Data 8910
Get Data 8911
Get Data 8912
Get Data 8913
Get Data 8914
Get Data 8915
Get Data 8916
Get Data 8917
Get Data 8918
Get Data 8919
Get Data 8920
Get Data 8921
Get Data 8922
Get Data 8923
Get Data 8924
Get Data 8925
Get Data 8926
Get Data 8927
Get Data 8928
Get Data 8929
Get Data 8930
Get Data 8931
Get Data 8932
Get Data 8933
Get Data 8934
Get Data 8935
Get Data 8936
Get Data 8937
Get Data 8938
Get Data 8939
Get Da

Get Data 9455
Get Data 9456
Get Data 9457
Get Data 9458
Get Data 9459
Get Data 9460
Get Data 9461
Get Data 9462
Get Data 9463
Get Data 9464
Get Data 9465
Get Data 9466
Get Data 9467
Get Data 9468
Get Data 9469
Get Data 9470
Get Data 9471
Get Data 9472
Get Data 9473
Get Data 9474
Get Data 9475
Get Data 9476
Get Data 9477
Get Data 9478
Get Data 9479
Get Data 9480
Get Data 9481
Get Data 9482
Get Data 9483
Get Data 9484
Get Data 9485
Get Data 9486
Get Data 9487
Get Data 9488
Get Data 9489
Get Data 9490
Get Data 9491
Get Data 9492
Get Data 9493
Get Data 9494
Get Data 9495
Get Data 9496
Get Data 9497
Get Data 9498
Get Data 9499
Get Data 9500
Get Data 9501
Get Data 9502
Get Data 9503
Get Data 9504
Get Data 9505
Get Data 9506
Get Data 9507
Get Data 9508
Get Data 9509
Get Data 9510
Get Data 9511
Get Data 9512
Get Data 9513
Get Data 9514
Get Data 9515
Get Data 9516
Get Data 9517
Get Data 9518
Get Data 9519
Get Data 9520
Get Data 9521
Get Data 9522
Get Data 9523
Get Data 9524
Get Data 9525
Get Da

Get Data 10038
Get Data 10039
Get Data 10040
Get Data 10041
Get Data 10042
Get Data 10043
Get Data 10044
Get Data 10045
Get Data 10046
Get Data 10047
Get Data 10048
Get Data 10049
Get Data 10050
Get Data 10051
Get Data 10052
Get Data 10053
Get Data 10054
Get Data 10055
Get Data 10056
Get Data 10057
Get Data 10058
Get Data 10059
Get Data 10060
Get Data 10061
Get Data 10062
Get Data 10063
Get Data 10064
Get Data 10065
Get Data 10066
Get Data 10067
Get Data 10068
Get Data 10069
Get Data 10070
Get Data 10071
Get Data 10072
Get Data 10073
Get Data 10074
Get Data 10075
Get Data 10076
Get Data 10077
Get Data 10078
Get Data 10079
Get Data 10080
Get Data 10081
Get Data 10082
Get Data 10083
Get Data 10084
Get Data 10085
Get Data 10086
Get Data 10087
Get Data 10088
Get Data 10089
Get Data 10090
Get Data 10091
Get Data 10092
Get Data 10093
Get Data 10094
Get Data 10095
Get Data 10096
Get Data 10097
Get Data 10098
Get Data 10099
Get Data 10100
Get Data 10101
Get Data 10102
Get Data 10103
Get Data 1

Get Data 10585
Get Data 10586
Get Data 10587
Get Data 10588
Get Data 10589
Get Data 10590
Get Data 10591
Get Data 10592
Get Data 10593
Get Data 10594
Get Data 10595
Get Data 10596
Get Data 10597
Get Data 10598
Get Data 10599
Get Data 10600
Get Data 10601
Get Data 10602
Get Data 10603
Get Data 10604
Get Data 10605
Get Data 10606
Get Data 10607
Get Data 10608
Get Data 10609
Get Data 10610
Get Data 10611
Get Data 10612
Get Data 10613
Get Data 10614
Get Data 10615
Get Data 10616
Get Data 10617
Get Data 10618
Get Data 10619
Get Data 10620
Get Data 10621
Get Data 10622
Get Data 10623
Get Data 10624
Get Data 10625
Get Data 10626
Get Data 10627
Get Data 10628
Get Data 10629
Get Data 10630
Get Data 10631
Get Data 10632
Get Data 10633
Get Data 10634
Get Data 10635
Get Data 10636
Get Data 10637
Get Data 10638
Get Data 10639
Get Data 10640
Get Data 10641
Get Data 10642
Get Data 10643
Get Data 10644
Get Data 10645
Get Data 10646
Get Data 10647
Get Data 10648
Get Data 10649
Get Data 10650
Get Data 1

Get Data 11132
Get Data 11133
Get Data 11134
Get Data 11135
Get Data 11136
Get Data 11137
Get Data 11138
Get Data 11139
Get Data 11140
Get Data 11141
Get Data 11142
Get Data 11143
Get Data 11144
Get Data 11145
Get Data 11146
Get Data 11147
Get Data 11148
Get Data 11149
Get Data 11150
Get Data 11151
Get Data 11152
Get Data 11153
Get Data 11154
Get Data 11155
Get Data 11156
Get Data 11157
Get Data 11158
Get Data 11159
Get Data 11160
Get Data 11161
Get Data 11162
Get Data 11163
Get Data 11164
Get Data 11165
Get Data 11166
Get Data 11167
Get Data 11168
Get Data 11169
Get Data 11170
Get Data 11171
Get Data 11172
Get Data 11173
Get Data 11174
Get Data 11175
Get Data 11176
Get Data 11177
Get Data 11178
Get Data 11179
Get Data 11180
Get Data 11181
Get Data 11182
Get Data 11183
Get Data 11184
Get Data 11185
Get Data 11186
Get Data 11187
Get Data 11188
Get Data 11189
Get Data 11190
Get Data 11191
Get Data 11192
Get Data 11193
Get Data 11194
Get Data 11195
Get Data 11196
Get Data 11197
Get Data 1

In [8]:
len(geodata)

11267

In [51]:
#Create - Empty List to store the Data that we need from JSON geodata

county_list = []
city_list = []
district_list = []

#Loop Through - to fulfill the Data List

for i in range(0,11268):
    county_list.append(geodata[i]['items'][0]['address']['county'])
    city_list.append(geodata[i]['items'][0]['address']['city'])
    district_list.append(geodata[i]['items'][0]['address']['district'])

In [49]:
rdf_revgeo= pd.DataFrame({'Province': county_list, 'City': city_list, 'District/Kecamatan': district_list })

In [47]:
rdf_revgeo

,Province,City,District/Kecamatan
0,DKI Jakarta,Jakarta Selatan,Kebayoran Baru
1,DKI Jakarta,Jakarta Pusat,Menteng
2,DKI Jakarta,Jakarta Pusat,Menteng
3,DKI Jakarta,Jakarta Pusat,Menteng
4,Jawa Barat,Bekasi Kota,Bekasi Selatan
...,...,...,...
10785,DKI Jakarta,Jakarta Selatan,Setiabudi
10786,Jawa Barat,Bekasi Kota,Medan Satria
10787,Banten,Tangerang Kabupaten,Pagedangan
10788,DKI Jakarta,Jakarta Utara,Pademangan


In [70]:
#Make A New DataFrame - Combine Original DataFrame with ReverseGocoing DataFrame

rdf_geo_comp = pd.merge(left=rdf_geo, right=rdf_revgeo, left_index=True, right_index=True) 

In [72]:
rdf_geo_comp.drop(['index'], axis=1, inplace=True)

In [73]:
rdf_geo_comp.to_excel('Zomato Restaurant DataFrame - Reverse Geocoding.xlsx', index = False)
rdf_geo_comp.to_csv('Zomato Restaurant DataFrame - Reverse Geocoding.csv', index = False)

# STEP 3 - CORRECTING SOME IRREGULAR DATA

In [2]:
rdf_geo_comp = pd.read_csv('Zomato Restaurant DataFrame - Reverse Geocoding.csv', index_col = False)

## PART 1 - "STRANGE" PROVINCE

In [4]:
#Recheck the Unique Values of Province, we see 'Jawa Timur' which is very far from Jakarta

rdf_geo_comp['Province'].unique()

array(['DKI Jakarta', 'Jawa Barat', 'Banten', 'Jawa Timur'], dtype=object)

In [5]:
rdf_geo_comp[rdf_geo_comp['Province'] == 'Jawa Timur']

,Restaurant Name,Restaurant Area,Restaurant Address,Latitude,Longitude,Province,City,District/Kecamatan
8933,Warung Kopi Konco Lawas,Serpong,"Ruko Puspita Loka, Jl. Anggrek Hitam Blok AF N...",-7.247999,112.777749,Jawa Timur,Surabaya,Tambaksari


After checking the Zomato Website, we saw that the Maps for Warung Kopi Konco Lawas is indeed wrong, showing a place in Surabaya,
instead of Serpong.

Let's do a quick search on Google Maps to get the exact Latitude and Longitude for this specific restaurant.

After we get the value, let's change it directly into the DataFrame

In [34]:
rdf_geo_comp.at[8933,'Latitude'] = -6.301646
rdf_geo_comp.at[8933,'Longitude'] = 106.6712919

After we get the Latitude and Longitude, we can use HERE Maps Reverse Geocoding again to get the accurate data

In [36]:
geodata

[{'items': [{'title': '32, Jalan Anggrek Hitam, Tangerang Selatan 15318, Indonesia',
    'id': 'here:af:streetsection:NYu39c1O62TmPN7Hg.zQkA:CggIBCDy37T8AhABGgIzMg',
    'resultType': 'houseNumber',
    'houseNumberType': 'PA',
    'address': {'label': '32, Jalan Anggrek Hitam, Tangerang Selatan 15318, Indonesia',
     'countryCode': 'IDN',
     'countryName': 'Indonesia',
     'state': 'JABODETABEK',
     'county': 'Banten',
     'city': 'Tangerang Selatan',
     'district': 'Serpong',
     'subdistrict': 'Rawa Buntu Kel.',
     'street': 'Jalan Anggrek Hitam',
     'postalCode': '15318',
     'houseNumber': '32'},
    'position': {'lat': -6.30173, 'lng': 106.67135},
    'access': [{'lat': -6.3017, 'lng': 106.67151}],
    'distance': 11,
    'mapView': {'west': 106.67104,
     'south': -6.3035,
     'east': 106.67194,
     'north': -6.29934}}]}]

In [38]:
rdf_geo_comp.at[8933,'Province'] = 'Banten'
rdf_geo_comp.at[8933,'City'] = 'Tangerang Selatan'
rdf_geo_comp.at[8933,'District/Kecamatan'] = 'Serpong'

In [40]:
#Data has been corrected. Jawa Timur is no more.

rdf_geo_comp.loc[8933]

Restaurant Name                                 Warung Kopi Konco Lawas
Restaurant Area                                                 Serpong
Restaurant Address    Ruko Puspita Loka, Jl. Anggrek Hitam Blok AF N...
Latitude                                                       -6.30165
Longitude                                                       106.671
Province                                                         Banten
City                                                  Tangerang Selatan
District/Kecamatan                                              Serpong
Name: 8933, dtype: object

## PART 2 - INCONSISTENT CITY

In [45]:
#Recheck the Unique Values of Cities

rdf_geo_comp['City'].unique()

array(['Jakarta Selatan', 'Jakarta Pusat', 'Bekasi Kota',
       'Tangerang Selatan', 'Depok', 'Tangerang Kota', 'Jakarta Utara',
       'Jakarta Barat', 'Jakarta Timur', 'Bogor Kota', 'Bogor Kabupaten',
       'Bekasi Kabupaten', 'Tangerang Kabupaten', 'Jakarta', 'Cianjur',
       'Bekasi', 'Tangerang'], dtype=object)

We can see that apart from Jakarta Utara, Selatan, Pusat, Timur, Barat, there is also "Jakarta"

This is inconsistent with previous categories. We have to impute this category with another category so we can make better Data Visualization and Modeling in the next process

In [47]:
rdf_geo_comp[rdf_geo_comp['City'] == 'Jakarta']

,Restaurant Name,Restaurant Area,Restaurant Address,Latitude,Longitude,Province,City,District/Kecamatan
64,d'Cobek,Bintaro,"D' Hoek, Jl. Bintaro Utama, Bintaro, Jakarta",-6.271864,106.755711,DKI Jakarta,Jakarta,Pesanggrahan
396,Fudgy Bro,Bintaro,"Jl. Garuda, Bintaro, Jakarta",-6.268627,106.756677,DKI Jakarta,Jakarta,Pesanggrahan
570,Roasted Bar Jakarta,Bintaro,"Jl. Taman Bintaro No. 1, Bintaro, Jakarta",-6.271686,106.758049,DKI Jakarta,Jakarta,Pesanggrahan
988,Kopi Duo,Bintaro,"Gerbang Tol Veteran, Jl. Bhakti No. 27, Bintar...",-6.259665,106.765665,DKI Jakarta,Jakarta,Pesanggrahan
1091,Balanga Dapur Manado,Bintaro,"Jl. Kesehatan Raya No. 11, Bintaro, Jakarta",-6.271035,106.757992,DKI Jakarta,Jakarta,Pesanggrahan
1967,Shini Mura,Bintaro,"Warung Jajan Veteran, Jl. RC Veteran Raya No. ...",-6.274959,106.762862,DKI Jakarta,Jakarta,Pesanggrahan
2107,Pempek Bang Je,Bintaro,"Warung Jajan Veteran, Jl. RC Veteran No. 6, Bi...",-6.274964,106.762856,DKI Jakarta,Jakarta,Pesanggrahan
2165,Barokah,Bintaro,"Warung Jajan Veteran, Jl. RC Veteran No. 6, Bi...",-6.272834,106.764091,DKI Jakarta,Jakarta,Pesanggrahan
2301,Bendino Cafea,Bintaro,"Jl. IKPN No.4, Bintaro, Jakarta",-6.260090,106.769976,DKI Jakarta,Jakarta,Pesanggrahan
2566,Kunanti Kopi,Bintaro,"Jl. Murai Raya, Bintaro Sektor 1, Blok J1 No. ...",-6.270046,106.754768,DKI Jakarta,Jakarta,Pesanggrahan


All Entries are located in District/Kecamatan Pesanggrahan.

Let's check with other data from Pesanggrahan.

In [43]:
rdf_geo_comp[rdf_geo_comp['District/Kecamatan'] == 'Pesanggrahan']['City'].unique()

array(['Jakarta', 'Jakarta Selatan'], dtype=object)

We can see that Kecamatan Pesanggrahan is actually located in Jakarta Selatan, so let's impute previous inconsistent data with Jakarta Selatan

In [48]:
tojaksel = rdf_geo_comp[rdf_geo_comp['City'] == 'Jakarta'].index

In [50]:
for i in tojaksel:
    rdf_geo_comp.at[i,'City'] = 'Jakarta Selatan'

In [53]:
#Let's check the result

rdf_geo_comp.loc[tojaksel]

,Restaurant Name,Restaurant Area,Restaurant Address,Latitude,Longitude,Province,City,District/Kecamatan
64,d'Cobek,Bintaro,"D' Hoek, Jl. Bintaro Utama, Bintaro, Jakarta",-6.271864,106.755711,DKI Jakarta,Jakarta Selatan,Pesanggrahan
396,Fudgy Bro,Bintaro,"Jl. Garuda, Bintaro, Jakarta",-6.268627,106.756677,DKI Jakarta,Jakarta Selatan,Pesanggrahan
570,Roasted Bar Jakarta,Bintaro,"Jl. Taman Bintaro No. 1, Bintaro, Jakarta",-6.271686,106.758049,DKI Jakarta,Jakarta Selatan,Pesanggrahan
988,Kopi Duo,Bintaro,"Gerbang Tol Veteran, Jl. Bhakti No. 27, Bintar...",-6.259665,106.765665,DKI Jakarta,Jakarta Selatan,Pesanggrahan
1091,Balanga Dapur Manado,Bintaro,"Jl. Kesehatan Raya No. 11, Bintaro, Jakarta",-6.271035,106.757992,DKI Jakarta,Jakarta Selatan,Pesanggrahan
1967,Shini Mura,Bintaro,"Warung Jajan Veteran, Jl. RC Veteran Raya No. ...",-6.274959,106.762862,DKI Jakarta,Jakarta Selatan,Pesanggrahan
2107,Pempek Bang Je,Bintaro,"Warung Jajan Veteran, Jl. RC Veteran No. 6, Bi...",-6.274964,106.762856,DKI Jakarta,Jakarta Selatan,Pesanggrahan
2165,Barokah,Bintaro,"Warung Jajan Veteran, Jl. RC Veteran No. 6, Bi...",-6.272834,106.764091,DKI Jakarta,Jakarta Selatan,Pesanggrahan
2301,Bendino Cafea,Bintaro,"Jl. IKPN No.4, Bintaro, Jakarta",-6.260090,106.769976,DKI Jakarta,Jakarta Selatan,Pesanggrahan
2566,Kunanti Kopi,Bintaro,"Jl. Murai Raya, Bintaro Sektor 1, Blok J1 No. ...",-6.270046,106.754768,DKI Jakarta,Jakarta Selatan,Pesanggrahan


In [57]:
rdf_geo_comp.to_excel('Zomato Restaurant DataFrame - Reverse Geocoding 2.xlsx', index = False)
rdf_geo_comp.to_csv('Zomato Restaurant DataFrame - Reverse Geocoding 2.csv', index = False)